In [1]:
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pickle

In [2]:
ahu = pd.read_csv("porest_area_ahu_automatic.csv")
chiller = pd.read_csv("porest_area_chiller_automatic.csv")

column_e = ['no', 'id', 'seq', 'type', 'val', 'rv', 'ri', 'sv', 'si', 'tv', 'ti', 'fire', 'temperature', 'humidity', 'cdate']
e = pd.read_csv('porest_device_e_log.csv', names=column_e)

column_plant = ["no", "id", "seq", "air", "monoxide", "dioxide", "pm2d5", "pm10", "temperature", "humidity", "score", "cdate"]
plant = pd.read_csv('porest_device_plant_log.csv', names=column_plant)

# 장치 정보
devices = pd.read_excel("porest_devices.xlsx",header=1)

C:\Users\lotto\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# ahu 전처리, A 구역 없앤뒤
ahu_a = ahu[ahu['building'] == 'A']
ahu_a.drop(columns=['no', 'occupancy', 'out_temp', 'in_temp', 'zone_min_temp','building'], inplace=True)
# 원래 ahu 칼럼에 있던 관측치를 기존제어라고 명명
ahu_a['how_0'] = '기존제어'

ahu_a['cdate'] = pd.to_datetime(ahu_a['cdate'])

ahu_a

C:\Users\lotto\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/2107956963.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ahu_a['how_0'] = '기존제어'
C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/2107956963.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,ahu_no,power,inv,cdate,how_0
0,1,1,30,2023-01-31 10:09:00,기존제어
1,2,0,0,2023-01-31 10:09:00,기존제어
2,3,1,30,2023-01-31 10:09:00,기존제어
3,4,0,0,2023-01-31 10:09:00,기존제어
4,5,1,30,2023-01-31 10:09:00,기존제어
...,...,...,...,...,...
28348,5,1,40,2023-08-01 16:00:00,기존제어
28349,7,1,35,2023-08-01 16:00:00,기존제어
28350,8,1,40,2023-08-01 16:00:00,기존제어
28351,9,1,40,2023-08-01 16:00:00,기존제어


In [4]:
alog = pd.read_csv("porest_area_log.csv", usecols=[2, 3, 4, 5, 6, 7],
                    names=['type', 'mode', 'text', 'for_schedule_control', 'cdate_0', 'cdate_1'], header=None)
alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
alog = alog[(alog['type'] == 'freezer')|(alog['type'] == 'ahu')].reset_index(drop=True)
alog[['device', 'how_0', 'how_1', 'power']] = pd.DataFrame(alog.text.str.rsplit(',', 3).tolist())
alog[['name', 'day']] = pd.DataFrame(alog.device.str.rsplit(',', 1).tolist())

alog[['if_inv', 'inv_schedule']] = alog['for_schedule_control'].str.split(':', n=1, expand=True)
alog['inv_schedule'] = alog['inv_schedule'].fillna(0)
alog = alog.astype({'inv_schedule':'int'})

alog['device'] = alog['name'].str.replace(',','')
alog['cdate'] = np.where(pd.isna(alog['cdate_0']), alog['cdate_1'], alog['cdate_0'])
alog['cdate'] = pd.to_datetime(alog['cdate'])
#alog['cdate'] = alog['cdate'].dt.round('10min')

alog_filtered = alog[['type', 'mode', 'device', 'how_0', 'how_1', 'power', 'day', 'inv_schedule', 'cdate']]

# device를 name과 number로 분리
alog_filtered['dname'] = alog_filtered['device'].str.extract('([가-힣]+)')
alog_filtered['dnumber'] = alog_filtered['device'].str.extract('(\d+)')
alog_filtered['dnumber'] = alog_filtered['dnumber'].astype('int')
alog_filtered.drop('device', axis=1, inplace=True)

alog_filtered = alog_filtered[alog_filtered['how_0'] != '자동제어변경'] # 자동제어변경 제외

ahu_plus = alog_filtered[alog_filtered['type']=='ahu']
ahu_plus = ahu_plus.rename(columns={'dnumber':'ahu_no'})

ahu_plus = ahu_plus[ahu_plus['how_0'] != '강제제어']

ahu_plus = ahu_plus[ahu_plus['ahu_no'] <= 14].reset_index(drop=True)
ahu_plus.drop(columns=['type', 'day', 'dname', 'mode'], inplace=True)

ahu_plus['power'] = ahu_plus['power'].str.replace('Hz', '')
ahu_plus = ahu_plus.astype({'power':'int'})

ahu_plus['power_eda'] = np.where(((ahu_plus['how_0'] == '스케줄제어')&(ahu_plus['power'] == 1))|
                                 ((ahu_plus['how_0'] == '수동제어')&(ahu_plus['power'] > 0)), 1, 0)

ahu_plus['inv'] = np.where((ahu_plus['power_eda'] == 1)&(ahu_plus['power'] > 1), ahu_plus['power'], 0)
ahu_plus['inv'] = np.where((ahu_plus['how_0'] == '스케줄제어')&(ahu_plus['power_eda'] == 1),
                           ahu_plus['inv_schedule'], ahu_plus['inv'])


ahu_plus.drop(columns=['power', 'how_1'], inplace=True)
ahu_plus.rename(columns={'power_eda':'power'}, inplace=True)

ahu_plus

C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/2454286824.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  alog['text'] = alog['text'].str.replace('::',',').str.replace('[','').str.replace(']',',').str.replace('\\','').str.replace(':',',')
C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/2454286824.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alog_filtered['dname'] = alog_filtered['device'].str.extract('([가-힣]+)')
C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/2454286824.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,how_0,inv_schedule,cdate,ahu_no,power,inv
0,스케줄제어,35,2023-01-01 08:00:03,1,1,35
1,스케줄제어,35,2023-01-01 08:00:03,3,1,35
2,스케줄제어,35,2023-01-01 08:50:04,2,1,35
3,스케줄제어,35,2023-01-01 09:00:03,5,1,35
4,스케줄제어,35,2023-01-01 09:00:03,11,1,35
...,...,...,...,...,...,...
10708,수동제어,0,2023-08-01 08:49:09,10,1,45
10709,수동제어,0,2023-08-01 08:49:19,11,1,45
10710,수동제어,0,2023-08-01 08:49:28,12,1,45
10711,수동제어,0,2023-08-01 08:49:37,13,1,45


In [5]:
ahu_combine = pd.concat([ahu_a, ahu_plus]).sort_values(by='cdate').reset_index(drop=True)
ahu_combine['inv'] = np.where(ahu_combine['power'] == 0, 0, ahu_combine['inv'])

ahu_combine['cdate'] = ahu_combine['cdate'].dt.round('10min')

ahu_combine = ahu_combine[ahu_combine['cdate'] >= '2023-01-01 08:00:00']

start_date = '2023-01-01 08:00:00'
end_date = '2023-08-01 16:00:00'

date_range = pd.date_range(start=start_date, end=end_date, freq='10T')

ahu_dct = {}

for i in range(1, 15):
    df_datetime = pd.DataFrame(np.nan, columns=['ahu_no', 'power', 'inv', 'how_0'], index=date_range)
    
    ahu_df = ahu_combine[ahu_combine['ahu_no'] == i]
    ahu_df = ahu_df.drop_duplicates(['cdate'], keep='last')
    
    ahu_df.set_index('cdate', inplace=True)
    
    df_datetime.loc[ahu_df.index, :] = ahu_df[:]
    df_datetime.fillna(method='ffill', inplace=True)
    
    ahu_dct[i] = df_datetime


In [7]:
ahu_1 = ahu_dct[1]
ahu_1[(ahu_1['power'] == 1)&(ahu_1['inv'] == 0)].tail(10)

,ahu_no,power,inv,how_0
2023-07-03 18:20:00,1.0,1.0,0.0,수동제어
2023-07-03 18:30:00,1.0,1.0,0.0,수동제어
2023-07-03 18:40:00,1.0,1.0,0.0,수동제어
2023-07-03 18:50:00,1.0,1.0,0.0,수동제어
2023-07-03 19:00:00,1.0,1.0,0.0,수동제어
2023-07-03 19:10:00,1.0,1.0,0.0,수동제어
2023-07-03 19:20:00,1.0,1.0,0.0,수동제어
2023-07-03 19:30:00,1.0,1.0,0.0,수동제어
2023-07-03 19:40:00,1.0,1.0,0.0,수동제어
2023-07-03 19:50:00,1.0,1.0,0.0,수동제어


In [93]:
alog_filtered[alog_filtered['type'] == 'freezer']

,type,mode,how_0,how_1,power,day,cdate,dname,dnumber
19,freezer,1,스케줄제어,전원,1,일,2023-01-01 09:20:00,열원설비,2
85,freezer,1,스케줄제어,전원,0,일,2023-01-01 21:00:00,열원설비,2
106,freezer,1,스케줄제어,전원,1,월,2023-01-02 09:20:00,열원설비,2
134,freezer,1,스케줄제어,전원,0,월,2023-01-02 21:00:00,열원설비,2
158,freezer,1,스케줄제어,전원,1,화,2023-01-03 09:20:00,열원설비,2
...,...,...,...,...,...,...,...,...,...
20717,freezer,1,스케줄제어,전원,1,월,2023-07-31 08:00:00,열원설비,1
20744,freezer,0,수동제어,인버터,50Hz,None,2023-07-31 08:50:00,열원설비,1
20765,freezer,1,스케줄제어,전원,0,월,2023-07-31 21:10:00,열원설비,1
20792,freezer,1,스케줄제어,전원,1,화,2023-08-01 08:00:00,열원설비,1


In [6]:
chiller_auto = chiller[['chiller_no', 'power', 'inv', 'cdate']]
chiller_auto['how_0'] = '자동제어'

chiller_auto['cdate'] = pd.to_datetime(chiller_auto['cdate'])

chiller_passive = alog_filtered[alog_filtered['type'] == 'freezer']
chiller_passive['power'] = chiller_passive['power'].str.replace('Hz', '')
chiller_passive = chiller_passive.astype({'power':'int'})

chiller_passive['power_eda'] = np.where(((chiller_passive['how_0'] == '스케줄제어')&(chiller_passive['power'] == 1))|
                                 ((chiller_passive['how_0'] == '수동제어')&(chiller_passive['power'] > 0)), 1, 0)
chiller_passive['inv'] = np.where((chiller_passive['power_eda'] == 1)&(chiller_passive['power'] > 1), chiller_passive['power'], 0)

chiller_passive.drop(columns=['power', 'how_1'], inplace=True)
chiller_passive.rename(columns={'power_eda':'power', 'dnumber':'chiller_no'}, inplace=True)

chiller_passive = chiller_passive[['chiller_no', 'power', 'inv', 'cdate', 'how_0']]

C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/1004769904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chiller_auto['how_0'] = '자동제어'
C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/1004769904.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chiller_auto['cdate'] = pd.to_datetime(chiller_auto['cdate'])
C:\Users\lotto\AppData\Local\Temp/ipykernel_24680/1004769904.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [7]:
chiller_combine = pd.concat([chiller_auto, chiller_passive]).sort_values(by='cdate').reset_index(drop=True)
chiller_combine['inv'] = np.where(chiller_combine['power'] == 0, 0, chiller_combine['inv'])

chiller_combine['cdate'] = chiller_combine['cdate'].dt.round('10min')

chiller_combine = chiller_combine[chiller_combine['cdate'] >= '2023-01-01 08:00:00']

chiller_dct = {}

for i in range(1, 3):
    df_datetime = pd.DataFrame(np.nan, columns=['chiller_no', 'power', 'inv', 'how_0'], index=date_range)
    
    chiller_df = chiller_combine[chiller_combine['chiller_no'] == i]
    chiller_df = chiller_df.drop_duplicates(['cdate'], keep='last')
    
    chiller_df.set_index('cdate', inplace=True)
    
    df_datetime.loc[chiller_df.index, :] = chiller_df[:]
    df_datetime.fillna(method='ffill', inplace=True)
    
    chiller_dct[i] = df_datetime


In [9]:
chiller_dct[1].dropna()

,chiller_no,power,inv,how_0
2023-01-16 09:20:00,1.0,1.0,0.0,스케줄제어
2023-01-16 09:30:00,1.0,1.0,0.0,스케줄제어
2023-01-16 09:40:00,1.0,1.0,0.0,스케줄제어
2023-01-16 09:50:00,1.0,1.0,0.0,스케줄제어
2023-01-16 10:00:00,1.0,1.0,0.0,스케줄제어
...,...,...,...,...
2023-08-01 15:20:00,1.0,1.0,41.0,자동제어
2023-08-01 15:30:00,1.0,1.0,41.0,자동제어
2023-08-01 15:40:00,1.0,1.0,41.0,자동제어
2023-08-01 15:50:00,1.0,1.0,41.0,자동제어


In [10]:
plant = pd.read_csv('porest_device_plant_log.csv', usecols=[1, 3, 4, 5, 6, 7, 8, 9, 11], 
                    names=['id', 'air', 'monoxide', 'dioxide', 'pm2d5', 'pm10', 'temperature', 'humidity', 'cdate'], header=None)
plant['cdate'] = pd.to_datetime(plant['cdate'])

devices_plant = pd.read_excel('porest_devices.xlsx', 'PLANT', skiprows=[0])
plant_id = list(devices_plant.iloc[:10, 1])
plant_filtered = plant[(plant['id'].isin(plant_id))].reset_index(drop=True)

plant_filtered['id'] = plant_filtered['id'].replace(plant_id, list(range(1,11)))

plant_filtered.set_index('cdate', inplace=True)
plant_filtered.index = pd.to_datetime(plant_filtered.index)

plant_group = plant_filtered.groupby('id')

start_date = '2023-01-01 08:00:00'
end_date = '2023-08-01 16:00:00'

plant_dict = {}
for name, group_df in plant_group:
    group_df.index = group_df.index.floor('10T') # 기존의 datetime을 10분 단위로 내림
    # group_df = group_df.loc[~group_df.index.duplicated(keep='first')]
    group_df = group_df.loc[~group_df.index.duplicated(keep='first')] # 내리는 과정에서 중복 발생시 중복 제거

    date_range = pd.date_range(start=start_date, end=end_date, freq='10T')
    group_df = group_df.reindex(date_range) # 10분 단위로 row를 생성
    
    group_df = group_df.replace('\\N', np.NaN)
    
    group_df = group_df.astype({'air':'float', 'monoxide':'float', 'dioxide':'float', 'pm2d5':'float', 'pm10':'float'})
    #group_df = group_df.interpolate('index') #결측치 채움

    plant_dict[name] = group_df

C:\Users\lotto\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
df_all = ahu_dct[1][['inv']]
df_all = df_all.add_prefix('Ahu1_')

for i in range(2, 15):
    df_add = ahu_dct[i][['inv']]
    df_add = df_add.add_prefix('Ahu' + str(i) + '_')
    df_all = pd.concat([df_all, df_add],axis=1)

for i in range(1, 3):
    df_add = chiller_dct[i][['inv']]
    df_add = df_add.add_prefix('Chiller' + str(i) + '_')
    df_all = pd.concat([df_all, df_add],axis=1)
    
for i in range(1, 11):
    df_add = plant_dict[i].drop('id', axis=1)
    df_add = df_add.add_prefix('Plant' + str(i) + '_')
    df_all = pd.concat([df_all, df_add],axis=1)
    
df_all

,Ahu1_inv,Ahu2_inv,Ahu3_inv,Ahu4_inv,Ahu5_inv,Ahu6_inv,Ahu7_inv,Ahu8_inv,Ahu9_inv,Ahu10_inv,...,Plant9_pm10,Plant9_temperature,Plant9_humidity,Plant10_air,Plant10_monoxide,Plant10_dioxide,Plant10_pm2d5,Plant10_pm10,Plant10_temperature,Plant10_humidity
2023-01-01 08:00:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,23.09,51.27,54.0,0.0,441.0,39.0,44.0,23.36,50.24
2023-01-01 08:10:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0,23.49,50.67,94.0,1.0,480.0,37.0,41.0,23.73,50.04
2023-01-01 08:20:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.0,23.60,50.23,275.0,1.0,462.0,38.0,43.0,23.91,49.27
2023-01-01 08:30:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.0,23.76,49.85,0.0,0.0,458.0,37.0,41.0,23.99,48.96
2023-01-01 08:40:00,35.0,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,26.0,23.94,48.34,0.0,1.0,418.0,26.0,29.0,24.16,47.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 15:20:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,2.0,31.45,54.25,0.0,0.0,553.0,6.0,7.0,31.32,54.64
2023-08-01 15:30:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,5.0,31.45,54.15,0.0,2.0,534.0,6.0,6.0,31.23,54.74
2023-08-01 15:40:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,4.0,31.35,54.55,0.0,0.0,557.0,6.0,6.0,31.29,54.61
2023-08-01 15:50:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,4.0,31.35,54.71,0.0,2.0,544.0,5.0,5.0,31.15,55.20


In [14]:
df_all.fillna(method='bfill')

,Ahu1_inv,Ahu2_inv,Ahu3_inv,Ahu4_inv,Ahu5_inv,Ahu6_inv,Ahu7_inv,Ahu8_inv,Ahu9_inv,Ahu10_inv,...,Plant9_pm10,Plant9_temperature,Plant9_humidity,Plant10_air,Plant10_monoxide,Plant10_dioxide,Plant10_pm2d5,Plant10_pm10,Plant10_temperature,Plant10_humidity
2023-01-01 08:00:00,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,40.0,23.09,51.27,54.0,0.0,441.0,39.0,44.0,23.36,50.24
2023-01-01 08:10:00,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,35.0,23.49,50.67,94.0,1.0,480.0,37.0,41.0,23.73,50.04
2023-01-01 08:20:00,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,37.0,23.60,50.23,275.0,1.0,462.0,38.0,43.0,23.91,49.27
2023-01-01 08:30:00,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,37.0,23.76,49.85,0.0,0.0,458.0,37.0,41.0,23.99,48.96
2023-01-01 08:40:00,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,...,26.0,23.94,48.34,0.0,1.0,418.0,26.0,29.0,24.16,47.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01 15:20:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,2.0,31.45,54.25,0.0,0.0,553.0,6.0,7.0,31.32,54.64
2023-08-01 15:30:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,5.0,31.45,54.15,0.0,2.0,534.0,6.0,6.0,31.23,54.74
2023-08-01 15:40:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,4.0,31.35,54.55,0.0,0.0,557.0,6.0,6.0,31.29,54.61
2023-08-01 15:50:00,40.0,35.0,40.0,40.0,40.0,35.0,35.0,36.0,36.0,40.0,...,4.0,31.35,54.71,0.0,2.0,544.0,5.0,5.0,31.15,55.20


In [20]:
df_all.to_csv('EDA_complete_1208.csv')

In [131]:
ahu_test = ahu_dct[8]
ahu_test[(ahu_test.index.hour <= 6)&(ahu_test['inv'] > 0)].index.day.unique()

Int64Index([3, 7, 9], dtype='int64')